In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


# Dataset

https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset

In [ ]:
# mini_face_dataset = angry + disgust combined into frown
data_dir = Path('/home/gleb/programming/play/tiny_ml/mini_face_dataset')
batch_size = 32
img_height = 48
img_width = 48

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_dir / 'train'),
  seed=0,
  color_mode='grayscale',
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_dir / 'validation'),
  seed=0,
  color_mode='grayscale',
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Cast data to float
train_ds.map(lambda x, y: (tf.cast(x, tf.float32), y))
val_ds.map(lambda x, y: (tf.cast(x, tf.float32), y))

class_names = train_ds.class_names
print(class_names)

num_classes = len(class_names)
dict_ = {}
for i in range(num_classes):
    dict_[i] = class_names[i]

model_path = './saved_model_weights_v3/best_model'


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  1)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)


# Model from tf tutorial

https://www.tensorflow.org/tutorials/images/classification

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()


In [ ]:
epochs=15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


# Model from GSoC post

https://github.com/theevildoof/TinyML_OV7670

In [ ]:
from keras.layers.convolutional import DepthwiseConv2D
from keras.backend import relu
from keras.activations import softmax

model = keras.Sequential()
model.add(DepthwiseConv2D((3,3),input_shape=(img_height,img_width,1)))
# Convolution Layer 1
model.add(Conv2D(2, (3, 3)))      # 2 different 3x3 kernels -- so 2 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
convLayer01 = Activation('relu')                     # activation
model.add(convLayer01)

# Convolution Layer 2
model.add(Conv2D(2, (3, 3)))                        # 2 different 3x3 kernels -- so 2 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))                        # activation
convLayer02 = MaxPooling2D(pool_size=(2,2))          # Pool the max values over a 2x2 kernel
model.add(convLayer02)

# Convolution Layer 3
model.add(Conv2D(4,(3, 3)))                         # 4 different 3x3 kernels -- so 4 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
convLayer03 = Activation('relu')                     # activation
model.add(convLayer03)

# Convolution Layer 4
model.add(Conv2D(4, (3, 3)))                        # 4 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))                        # activation
convLayer04 = MaxPooling2D(pool_size=(2,2))          # Pool the max values over a 2x2 kernel
layers.Dropout(0.2)
model.add(convLayer04)

model.add(Flatten())
model.add(Dense(5,activation = relu))
model.add(Dense(num_classes, activation = softmax))

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()


In [ ]:
epochs=15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


# Model v3 (ours)

In [ ]:
from keras.layers.core import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten, AvgPool2D
from keras.layers import BatchNormalization, Dropout
from keras.layers.convolutional import DepthwiseConv2D
from keras.backend import relu
from keras.activations import softmax
from keras import regularizers

model = Sequential()

model.add(DepthwiseConv2D((3,3),input_shape=(img_width,img_height,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

model.add(Conv2D(2, (3, 3)))
model.add(BatchNormalization(axis=-1))
convLayer01 = Activation('relu')

model.add(Conv2D(2, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(convLayer01)
convLayer015 = AvgPool2D(pool_size=(2,2))
model.add(convLayer015)

model.add(Conv2D(4, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

model.add(Conv2D(4, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
convLayer02 = AvgPool2D(pool_size=(2,2))
model.add(convLayer02)

model.add(Conv2D(8,(3, 3)))
model.add(BatchNormalization(axis=-1))
convLayer03 = Activation('relu')

model.add(Conv2D(8,(3, 3)))
model.add(BatchNormalization(axis=-1))
convLayer03 = Activation('relu')
convLayer04 = AvgPool2D(pool_size=(2,2))
model.add(convLayer04)

model.add(Flatten())
model.add(Dense(5, activation = relu, kernel_regularizer=regularizers.L2(0.05)))
model.add(Dense(num_classes, activation = softmax))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.build((None, img_height, img_height, 1))
model.summary()


In [ ]:
epochs = 45
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


In [ ]:
model.save_weights(model_path)


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  predictions = model.predict(images)
  max_predictions = np.argmax(predictions, axis=1)
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(f'{dict_[max_predictions[i]]} ({class_names[labels[i]]})')
    plt.axis("off")
